<a href="https://colab.research.google.com/github/liliya111darky/logical-ai/blob/notebooks/05_veritas_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>